## Task

В качестве тестового задания мы предоставляем вам датасет online_shoppers_intention.csv, который содержит информацию о посетителях интернет-магазина и их поведении на сайте. Необходимо проанализировать данный датасет, определив какие действия могут привести к конверсии и создать классификатор на основе ваших гипотез.

В качестве ответа необходимо предоставить Jupyter Notebook с вашим анализом и модель машинного обучения, которая сможет предсказывать, приносят ли действия пользователя Revenue или нет.

## Imports

In [3]:
# Data
import pandas as pd
import numpy as np

# ML
import sklearn

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


## Data Analysis

In [4]:
dataframe = pd.read_csv('online_shoppers_intention.csv')
dataframe.sample(5)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
8719,0,0.000000,0,0.00,102,4509.000000,0.011765,0.037680,0.0,0.0,Nov,2,2,7,1,Returning_Visitor,False,False
1752,0,0.000000,0,0.00,44,1112.000000,0.000000,0.006818,0.0,0.0,Mar,4,1,4,3,Returning_Visitor,True,False
10281,4,111.166667,2,25.75,16,411.937500,0.009524,0.009524,0.0,0.0,Nov,3,2,8,2,New_Visitor,True,False
9798,0,0.000000,0,0.00,3,158.000000,0.033333,0.033333,0.0,0.0,Dec,1,1,1,3,New_Visitor,False,False
11904,1,27.500000,1,376.00,109,2925.677607,0.005505,0.015673,0.0,0.0,Nov,1,2,3,10,Returning_Visitor,False,False


Для большего понимания признаков был найден [источник](https://archive.ics.uci.edu/dataset/468/online+shoppers+purchasing+intention+dataset) данного датасета

Далее расписаны наиболее специфичные их них

"**Administrative**", "**Administrative Duration**", "**Informational**", "**Informational Duration**", "**Product Related**" и "**Product Related Duration**" представляют собой количество различных типов страниц, посещенных посетителем за один сеанс, и общее время, проведенное в каждой из этих категорий страниц.

Значение функции "**Bounce Rate**" для веб-страницы относится к проценту посетителей, которые заходят на сайт с этой страницы, а затем покидают ("bounce"), не инициируя никаких других запросов к серверу аналитики во время этого сеанса. 

Значение функции "**Exit Rate**" для конкретной веб-страницы рассчитывается как для всех просмотров страницы на странице процент, которые были последними в сеансе. 

Функция "**Page Value**" представляет собой среднюю ценность веб-страницы, которую пользователь посетил до завершения транзакции электронной торговли. 

Функция "**Special Day**" указывает на близость времени посещения сайта к определенному особому дню (например, Дню матери, Дню святого Валентина), когда сеансы с большей вероятностью завершатся транзакцией. Значение этого атрибута определяется с учетом динамики электронной торговли, такой как продолжительность между датой заказа и датой доставки. Например, для дня Валентины это значение принимает ненулевое значение между 2 и 12 февраля, ноль до и после этой даты, если только она не близка к другому особому дню, и максимальное значение 1 приходится на 8 февраля.

Рассмотрим есть ли пропущеные значения и каких типов бывают признаки в данном датасете

In [17]:
dataframe.info()

cat_cols = list(dataframe.select_dtypes(exclude='number').columns)
num_cols = list(dataframe.select_dtypes(include='number').columns)
print(f'All categorical features: {", ".join(cat_cols)}')
print(f'All numerical features: {", ".join(num_cols)}')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

Из полученных результатов видно, что нет пропущеных значений, а также имеется 8 категориальных признаков (Month, VisitorType, Weekend, Revenue, OperatingSystems, Browser, Region, TrafficType), из которых 4 последних уже закодированы LabelEncoder'ом.

In [9]:
dataframe.describe()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,OperatingSystems,Browser,Region,TrafficType
count,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000,12330.000000
mean,2.315166,80.818611,0.503569,34.472398,31.731468,1194.746220,0.022191,0.043073,5.889258,0.061427,2.124006,2.357097,3.147364,4.069586
std,3.321784,176.779107,1.270156,140.749294,44.475503,1913.669288,0.048488,0.048597,18.568437,0.198917,0.911325,1.717277,2.401591,4.025169
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,7.000000,184.137500,0.000000,0.014286,0.000000,0.000000,2.000000,2.000000,1.000000,2.000000
50%,1.000000,7.500000,0.000000,0.000000,18.000000,598.936905,0.003112,0.025156,0.000000,0.000000,2.000000,2.000000,3.000000,2.000000
75%,4.000000,93.256250,0.000000,0.000000,38.000000,1464.157214,0.016813,0.050000,0.000000,0.000000,3.000000,2.000000,4.000000,4.000000
max,27.000000,3398.750000,24.000000,2549.375000,705.000000,63973.522230,0.200000,0.200000,361.763742,1.000000,8.000000,13.000000,9.000000,20.000000


Из сводной таблици видно, что большинство численных признаков распределены в близи нуля, например Administrative,  Administrative_Duration, Informational, Informational_Duration, PageValues

Для более детального анализа рассмотрим каждый из признаков отдельно.